In [14]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigram = [((test_sentence[i], test_sentence[i + 1]), test_sentence[i + 2])
           for i in range(len(test_sentence) - 2)]

vocb = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}


class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size * n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out, dim=1)
        return log_prob


ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

for epoch in range(10):
    print('epoch: {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0
    for data in trigram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        running_loss += loss.data.item()
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))


epoch: 1
**********
Loss: 5.359205
epoch: 2
**********
Loss: 5.297864
epoch: 3
**********
Loss: 5.237424
epoch: 4
**********
Loss: 5.177791
epoch: 5
**********
Loss: 5.118863
epoch: 6
**********
Loss: 5.060462
epoch: 7
**********
Loss: 5.002414
epoch: 8
**********
Loss: 4.944770
epoch: 9
**********
Loss: 4.887314
epoch: 10
**********
Loss: 4.829735


In [15]:

for idx in range(len(trigram)):
    word, label = trigram[idx]
#     print(word)
#     print(label)

    word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
    out = ngrammodel(word)
    _, predict_label = torch.max(out, 1)

# # print(predict_label[0].item())
    predict_word = idx_to_word[predict_label.data[0].item()]
    print('real word is {}, predict word is {}'.format(label, predict_word))

real word is winters, predict word is thou
real word is shall, predict word is being
real word is besiege, predict word is besiege
real word is thy, predict word is thy
real word is brow,, predict word is much
real word is And, predict word is thou
real word is dig, predict word is thy
real word is deep, predict word is feel'st
real word is trenches, predict word is thy
real word is in, predict word is thy
real word is thy, predict word is thy
real word is beauty's, predict word is beauty's
real word is field,, predict word is thy
real word is Thy, predict word is thou
real word is youth's, predict word is thy
real word is proud, predict word is thy
real word is livery, predict word is livery
real word is so, predict word is when
real word is gazed, predict word is and
real word is on, predict word is thou
real word is now,, predict word is thou
real word is Will, predict word is being
real word is be, predict word is And
real word is a, predict word is And
real word is totter'd, predi

In [ ]:
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
